In [ ]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras import layers
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import backend as K


df = pd.read_csv('LBW_Dataset.csv')
df.Education.replace(np.NaN, 5, inplace=True)
df.Residence.fillna(df.Residence.mode()[0], inplace=True)
df["Delivery phase"].fillna(df["Delivery phase"].mode()[0], inplace=True)
df["Age"].fillna(df["Age"].median(axis=0), inplace=True)
df["Weight"].fillna(df["Weight"].mean(axis=0), inplace=True)
df["HB"].fillna(df["HB"].mean(axis=0), inplace=True)
df["BP"].fillna(df["BP"].mean(axis=0), inplace=True)
# print(df)


In [ ]:
df.groupby('Result').count()

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Education,Residence
Result,,,,,,,,,
0,24,24,24,24,24,24,24,24,24
1,72,72,72,72,72,72,72,72,72


In [ ]:
def sampling_k_elements(group, k=24):
    if len(group) < k:
        return group
    return group.sample(k)

balanced1 = df.loc[df['Result'] == 0].groupby('Result').apply(sampling_k_elements, 24).reset_index(drop=True)
balanced2 = df.loc[df['Result'] == 1].groupby('Result').apply(sampling_k_elements, 36).reset_index(drop=True)
balanced = pd.concat([balanced2, balanced1])
balanced.head()

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Education,Residence,Result
0,3,23.0,44.0,1.0,10.400000,1,1.375000,5.0,1.0,1
1,4,21.0,55.0,1.0,8.000000,1,1.571429,5.0,2.0,1
2,1,21.0,50.0,1.0,9.200000,1,1.625000,5.0,1.0,1
3,1,25.0,52.0,1.0,9.076623,0,1.725189,5.0,1.0,1
4,1,26.0,44.0,1.0,9.076623,0,1.725189,5.0,1.0,1


In [ ]:
df = balanced
balanced.shape

(60, 10)

In [ ]:
data = []


for i in range(len(df)):
    data.append({
            # "evidence": [int(df.iloc[i,0]), int(df.iloc[i,1]), int(df.iloc[i,2]), int(df.iloc[i,3]), float(df.iloc[i,4]), int(df.iloc[i,5]), float(df.iloc[i, 6]), int(df.iloc[i,7]), int(df.iloc[i, 8])],
            "evidence": [int(df.iloc[i,0]), int(df.iloc[i,1]), float(df.iloc[i,2]), int(df.iloc[i,3]), float(df.iloc[i,4]), int(df.iloc[i,5]), float(df.iloc[i, 6]), int(df.iloc[i, 8])],
            "label": int(df.iloc[i,9])
        })

In [ ]:
# ax1 = df.plot.scatter(x='Result',
#                       y='Delivery phase',  # 'Age' 'Community' 'Delivery phase' 'Weight' 'HB' 'IFA' 'BP'
#                       c='DarkBlue')

In [ ]:
df['Community'].unique()  # df['Delivery phase'].unique()

array([3, 4, 1, 2])

In [ ]:
df.loc[df['Delivery phase'] == 1].groupby('Result').count()

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Education,Residence
Result,,,,,,,,,
0,23,23,23,23,23,23,23,23,23
1,35,35,35,35,35,35,35,35,35


In [ ]:
df.loc[df['Delivery phase'] == 2].groupby('Result').count()

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Education,Residence
Result,,,,,,,,,
0,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1


In [ ]:
df.loc[df['Community'] == 4].groupby('Result').count()

,Community,Age,Weight,Delivery phase,HB,IFA,BP,Education,Residence
Result,,,,,,,,,
0,4,4,4,4,4,4,4,4,4
1,7,7,7,7,7,7,7,7,7


In [ ]:
# data

[{'evidence': [3, 23, 44.0, 1, 10.4, 1, 1.375, 1], 'label': 1},
 {'evidence': [4, 21, 55.0, 1, 8.0, 1, 1.571428571, 2], 'label': 1},
 {'evidence': [1, 21, 50.0, 1, 9.2, 1, 1.625, 1], 'label': 1},
 {'evidence': [1, 25, 52.0, 1, 9.076623376623376, 0, 1.725189285259259, 1],
  'label': 1},
 {'evidence': [1, 26, 44.0, 1, 9.076623376623376, 0, 1.725189285259259, 1],
  'label': 1},
 {'evidence': [3, 27, 45.0, 1, 9.076623376623376, 0, 1.5, 1], 'label': 1},
 {'evidence': [1, 26, 43.0, 1, 9.0, 1, 1.375, 1], 'label': 1},
 {'evidence': [4, 21, 48.0, 1, 9.2, 1, 1.375, 1], 'label': 1},
 {'evidence': [1, 24, 48.0, 1, 9.6, 1, 1.571428571, 1], 'label': 1},
 {'evidence': [1, 25, 50.0, 1, 5.9, 1, 1.428571429, 2], 'label': 1},
 {'evidence': [1, 24, 40.0, 1, 9.5, 1, 1.714285714, 1], 'label': 1},
 {'evidence': [1, 24, 39.0, 1, 8.9, 1, 1.375, 1], 'label': 1},
 {'evidence': [1, 24, 48.0, 1, 9.0, 1, 1.375, 1], 'label': 1},
 {'evidence': [1, 21, 40.0, 1, 9.2, 1, 1.571, 1], 'label': 1},
 {'evidence': [3, 20, 43.

In [ ]:
# Separate data into training and testing groups

evidence = [row["evidence"] for row in data]
labels = [row["label"] for row in data]
X_training, X_testing, y_training, y_testing = train_test_split(
    evidence, labels, test_size=0.25, random_state=42, stratify=labels
)

In [ ]:
sum([1 for x in y_training if x == 1]), sum([1 for x in y_training if x == 0])

(27, 18)

In [ ]:
# Create a neural network
model = tf.keras.models.Sequential([

    # tf.keras.layers.Dense(10, input_shape=(9,)),
    # tf.keras.layers.LeakyReLU(alpha=0.1),
    # tf.keras.layers.Dropout(0.25),

    # tf.keras.layers.Dense(5, input_shape=(16,)),
    # tf.keras.layers.LeakyReLU(alpha=0.1),
    # tf.keras.layers.Dropout(0.25),

    layers.Dense(8, activation="relu", name="layer1"),
    layers.Dropout(0.1),
    layers.Dense(2, activation="relu", name="layer2"),
    layers.Dropout(0.1),
    layers.Dense(1, activation="softmax", name="layer3")
])

In [ ]:
# Define our custom loss function
def focal_loss(y_true, y_pred):
    gamma = 1
    alpha = 0.1
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

# Compile our model
adam = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss=[focal_loss], metrics=["accuracy"], optimizer=adam) 

In [ ]:
# opt = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# # opt = tf.keras.optimizers.Adam(learning_rate=0.05)

# # Train neural network
# model.compile(
#     optimizer= opt,
#     loss="binary_crossentropy",
#     metrics=["accuracy"]
# )

In [ ]:
model.fit(X_training, y_training, epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6667
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.4000
Epoch 3/10
2/2 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.4000
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.4000
Epoch 5/10
2/2 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.4000
Epoch 6/10
2/2 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.4000
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.4000
Epoch 8/10
2/2 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.4000
Epoch 9/10
2/2 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.4000
Epoch 10/10
2/2 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.4000


In [ ]:
# Evaluate how well model performs
model.evaluate(X_testing, y_testing, verbose=2)

1/1 - 0s - loss: nan - accuracy: 0.4000


[nan, 0.4000000059604645]

In [ ]:
ynew = model.predict_classes(X_testing)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:457: RuntimeWarning: invalid value encountered in greater
  return (proba > 0.5).astype('int32')


In [ ]:
ynew

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=int32)

In [ ]:
y_testing

[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0]

In [ ]:
tf.math.confusion_matrix(y_testing, ynew, 2)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[6, 0],
       [9, 0]], dtype=int32)>